# Lecture 10 Neural Machine Translation and Models with Attention

## 什么是神经机器翻译（NMT）？

NMT是一个端到端的神经网络，不仅仅是nn+MT，通常是encoder-decoder的结构

![截屏2020-02-1115.57.36.png](https://i.loli.net/2020/02/11/dLFpWlymc9oaUju.png)

## RNN Encoder

最常见的方法是使用门控循环单元（gated recurrent unit），比如RNN或LSTM作为encoder

将encode得到的y作为hidden layer的起点，进行decode

![截屏2020-02-1217.18.01.png](https://i.loli.net/2020/02/12/afG9OB6Nhjck5u8.png)

如上图所示，所有encode的feature被表示为y，即图中的蓝色部分，这就要求遗忘门（forget gate）不要遗忘太多信息，以确保decode时有足够的信息

## Decoder: Recurrent Language Model

将source sentence的表示y输入到每个单词的生成中，所以生成每一个词的输入就是$x_t$和y

![截屏2020-02-1217.29.27.png](https://i.loli.net/2020/02/12/2ISM7XE3nkBF9bz.png)

## NMT的四大优点

1. 端到端训练：所有参数可以通过最小化网络输出的loss来进行最优化
2. 不同地方的词可以共享表示，以更好地应用词与段的相似性
3. 更好的利用上下文：传统方法对于每个词的预测是独立的，但NMT可以利用到原文本以及部分目标文本的上下文
4. 生成更流利的自然语言

## Attention

由于输入的表示仅有y，所以可能会导致对长句子的decode效果差，所以想要引入cv中attention的思想用于解决这一问题。既然encoder最后一层的y所包含的信息可能不足，我们就可以保存hidden layer中每一层的信息，在decode的时候选取我们需要的state，这就需要用到attention。

attention与对齐模型有些相似，因为即便是人类翻译人员在翻译的时候也需要将翻译文本与原文本进行对照，在静态的phrase-based模型中一个工作就是将两个语言对应的短语进行对齐，使用attention模型可以达到相似的目的，从而建立源语言与目标语言的联系

![截屏2020-02-1420.40.04.png](https://i.loli.net/2020/02/14/tU3AwFpxvl1NMry.png)

当我们对一个词进行预测时，不仅仅要利用前一个词的state（如上图红色部分所示），还要利用encoder的hidden state（上图蓝色部分），那么使用哪个hidden state呢？就需要我们用一个score function对这些state进行打分，然后将打分结果以权重的形式将这些hidden state结合起来，再过一个softmax，得到将attention放在这些hidden state的概率，再将这些hidden state结合起来作为context vector

![截屏2020-02-1420.46.53.png](https://i.loli.net/2020/02/14/rzve8oTI3l7jE51.png)

分数的获得，是通过attention function进行的。attention function有多种选择，其中流行的是中间这种。$W_a$给了两个向量更复杂的interaction，而最后一种根本没有interaction

![截屏2020-02-1420.53.51.png](https://i.loli.net/2020/02/14/miIBtLo3vE8AhQs.png)

## Global VS Local

![截屏2020-02-1421.06.12.png](https://i.loli.net/2020/02/14/cCMPgEztyi4lwXx.png)

实验发现，没有attention的模型效果随句子长度的增加会迅速下降，而即便是短句子，加了attention以后也会显著的提升效果

## 更多attention！覆盖范围

在图片标题生成研究中，模型通过对图片不同区域的attention生成了不同的词语：

![截屏2020-02-1421.44.44.png](https://i.loli.net/2020/02/14/bxD3E2OdQeik5Al.png)

如何保证不错过任何重要的区域呢？

一种思路是同时注意原文和译文（double attention）：

![截屏2020-02-1421.45.05.png](https://i.loli.net/2020/02/14/eO6jzB9iXFPpHmL.png)

还可以用旧模型的语言学思想拓展attention：

利用IBM2等模型中的位置或fertility(丰富程度)，因为一般而言一个词最多翻译为两三个词，如果生成了五六个词，那么模型可能在重复生成。

## Decoder

模型能够在给定原文的情况下计算译文的概率之后，就来到传统的问题了，找出最可能的译文。在decoding的时候，朴素想法是生成所有的翻译，用语言模型打分，然后挑最大的。但译文数量是词表大小的指数函数，无法实现。所以就产生了不同的选词策略

### Ancestral sampling

在时刻t，根据之前的词语生成当前词语$x_t$：

$x_{t} \sim \mathbb{P}\left(x_{t} | x_{1}, \ldots, x_{n}\right)$

可以多次sample取最好的。

理论上完美无缺，但实践中只会产生高方差的差效果。你也不想同一个句子每次翻译结果都不一样。

### Greedy Search

不多想了，贪婪地选取当前最可能的那个单词：

$x_{t}=\operatorname{argmax}_{\tilde{x}_{t}} \mathbb{P}\left(\tilde{x}_{t} | x_{1}, \ldots, x_{n}\right)$

![006Fmjmcly1fh19bsnpcuj309y0g2dh0.jpg](https://i.loli.net/2020/02/14/wzujPxp5kaWD8GT.jpg)

缺点显而易见，不是全局最优，走错一步会影响后面的部分。

### Beam search

老生常谈了，从不搜索到贪婪搜索到柱搜索，随处可见。每个时刻记录k个最可能的选项（剪枝），在其中进行搜索。

![006Fmjmcly1fh19citfnjj30cm0aeq4e.jpg](https://i.loli.net/2020/02/14/ywpsq8TV49QlIM3.jpg)

$\mathcal{H}_{t}=\left\{\left(x_{1}^{1}, \ldots, x_{t}^{1}\right), \ldots,\left(x_{1}^{K}, \ldots, x_{t}^{K}\right)\right\}$

则

$\tilde{\mathcal{H}}_{t+1}=\bigcup_{k=1}^{K} \mathcal{H}_{t+1}^{\widetilde{k}}$

其中

$\mathcal{H}_{t+1}^{\widetilde{\kappa}}=\left\{\left(x_{1}^{k}, \ldots, x_{t}^{k}, v_{1}\right), \ldots,\left(x_{1}^{k}, \ldots, x_{t}^{k}, v_{|V|}\right)\right\}$

也就是说把词表中的词丢进入计算概率取前K个。

### 效果对比

![006Fmjmcly1fh19j0rpu6j31960fognz.jpg](https://i.loli.net/2020/02/14/kKvBGpQEUcm1XMY.jpg)

采样要采50轮才得到比贪心搜索稍好的结果，但很小的柱搜索轻松超越了它们。另外，基于短语的MT常用的柱搜索大小是100到150，可见NMT的优势。
